#### Building OpenAI Agent using Llama-Index

Testing and exploring agents in llamaindex

In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

import warnings
warnings.filterwarnings("ignore")

In [2]:
import json
from typing import Sequence, List

from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

import nest_asyncio

nest_asyncio.apply()

#### for testing purpose let's define some very simple calculator tools for our agent

In [3]:
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

In [4]:
def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b


add_tool = FunctionTool.from_defaults(fn=add)

### Agent Definition

Now, we define our agent that’s capable of holding a conversation and calling tools in under 50 lines of code.

The meat of the agent logic is in the chat method. At a high-level, there are 3 steps:

Call OpenAI to decide which tool (if any) to call and with what arguments.

Call the tool with the arguments to obtain an output

Call OpenAI to synthesize a response from the conversation context and the tool output.

The reset method simply resets the conversation context, so we can start another conversation.

In [5]:
class YourOpenAIAgent:
    def __init__(
        self,
        tools: Sequence[BaseTool] = [],
        llm: OpenAI = OpenAI(temperature=0, model="gpt-3.5-turbo-0613"),
        chat_history: List[ChatMessage] = [],
    ) -> None:
        self._llm = llm
        self._tools = {tool.metadata.name: tool for tool in tools}
        self._chat_history = chat_history

    def reset(self) -> None:
        self._chat_history = []

    def chat(self, message: str) -> str:
        chat_history = self._chat_history
        chat_history.append(ChatMessage(role="user", content=message))
        tools = [
            tool.metadata.to_openai_tool() for _, tool in self._tools.items()
        ]

        ai_message = self._llm.chat(chat_history, tools=tools).message
        additional_kwargs = ai_message.additional_kwargs
        chat_history.append(ai_message)

        tool_calls = ai_message.additional_kwargs.get("tool_calls", None)
        # parallel function calling is now supported
        if tool_calls is not None:
            for tool_call in tool_calls:
                function_message = self._call_function(tool_call)
                chat_history.append(function_message)
                ai_message = self._llm.chat(chat_history).message
                chat_history.append(ai_message)

        return ai_message.content

    def _call_function(self, tool_call: dict) -> ChatMessage:
        id_ = tool_call["id"]
        function_call = tool_call["function"]
        tool = self._tools[function_call["name"]]
        output = tool(**json.loads(function_call["arguments"]))
        return ChatMessage(
            name=function_call["name"],
            content=str(output),
            role="tool",
            additional_kwargs={
                "tool_call_id": id_,
                "name": function_call["name"],
            },
        )
    

In [6]:
agent = YourOpenAIAgent(tools=[multiply_tool, add_tool])

In [7]:
agent.chat("Hi")

'Hello! How can I assist you today?'

#### Better way to implement an OpenAI Agent :

In [9]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI

In [10]:
llm = OpenAI(model="gpt-3.5-turbo-0613")
agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool], llm=llm, verbose=True
)

In [11]:
response = agent.chat("What is (121 * 3) + 42?")
print(str(response))

Added user message to memory: What is (121 * 3) + 42?
=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363

=== Calling Function ===
Calling function: add with args: {
  "a": 363,
  "b": 42
}
Got output: 405

(121 * 3) + 42 is equal to 405.


In [12]:
# inspect sources
print(response.sources)

[ToolOutput(content='363', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 121, 'b': 3}}, raw_output=363), ToolOutput(content='405', tool_name='add', raw_input={'args': (), 'kwargs': {'a': 363, 'b': 42}}, raw_output=405)]


#### Async Chat


In [13]:
response = await agent.achat("What is 121 * 3?")
print(str(response))

Added user message to memory: What is 121 * 3?
=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 3
}
Got output: 363

121 * 3 is equal to 363.


### Streaming Chat

In [14]:
response = agent.stream_chat(
    "What is 121 * 2? Once you have the answer, use that number to write a"
    " story about a group of mice."
)

response_gen = response.response_gen

for token in response_gen:
    print(token, end="")

Added user message to memory: What is 121 * 2? Once you have the answer, use that number to write a story about a group of mice.
=== Calling Function ===
Calling function: multiply with args: {
  "a": 121,
  "b": 2
}
Got output: 242

121 * 2 is equal to 242.

Once upon a time, in a small village, there was a group of mice who lived happily in a cozy little burrow. The leader of the group was a wise and courageous mouse named Milo. Milo was known for his intelligence and his ability to solve problems.

One day, Milo gathered all the mice together and announced that they were facing a shortage of food. The mice were worried and didn't know what to do. But Milo had a plan. He had heard about a bountiful cornfield located just a few miles away from their burrow.

With determination in his eyes, Milo rallied the mice and led them on a journey to the cornfield. They traveled through fields, crossed rivers, and climbed hills, all while staying together as a united group.

When they finally re

#### Async Streaming Chat


In [15]:
response = await agent.astream_chat(
    "What is 121 + 8? Once you have the answer, use that number to write a"
    " story about a group of mice."
)

response_gen = response.response_gen

async for token in response.async_response_gen():
    print(token, end="")

Added user message to memory: What is 121 + 8? Once you have the answer, use that number to write a story about a group of mice.
=== Calling Function ===
Calling function: add with args: {
  "a": 121,
  "b": 8
}
Got output: 129

121 + 8 is equal to 129.

Once upon a time, in a cozy little mouse hole, there lived a group of 129 mice. These mice were known for their intelligence and resourcefulness. They had built a thriving community within the walls of an old farmhouse.

Every day, the mice would venture out of their hole in search of food and adventure. They would scurry through the kitchen, carefully avoiding the watchful eyes of the cat. They would nibble on crumbs left behind by the humans and gather seeds from the pantry.

One day, as the mice were exploring a new part of the farmhouse, they stumbled upon a hidden treasure trove of cheese. It was a room filled with shelves stacked high with all kinds of cheese - cheddar, Swiss, Gouda, and even some exotic varieties.

The mice coul

#### Agent with Personality
You can specify a system prompt to give the agent additional instruction or personality.

In [16]:
from llama_index.agent.openai import OpenAIAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.prompts.system import SHAKESPEARE_WRITING_ASSISTANT

In [17]:
llm = OpenAI(model="gpt-3.5-turbo-0613")

agent = OpenAIAgent.from_tools(
    [multiply_tool, add_tool],
    llm=llm,
    verbose=True,
    system_prompt=SHAKESPEARE_WRITING_ASSISTANT,
)

In [18]:
response = agent.chat("Hi")
print(response)

Added user message to memory: Hi
Greetings, fair traveler! How may I assist thee on this fine day?


In [19]:
response = agent.chat("Tell me a story")
print(response)

Added user message to memory: Tell me a story
Of course, dear friend! Allow me to weave a tale for thee. Once upon a time, in a land far away, there lived a valiant knight named Sir William. He was known throughout the kingdom for his bravery and noble heart.

One fateful day, a messenger arrived at the castle with news of a fearsome dragon terrorizing the nearby village. The villagers were in great distress, for the dragon had been causing havoc and destruction wherever it went.

Sir William, hearing of their plight, knew he must embark on a quest to vanquish the dragon and bring peace back to the land. He donned his shining armor, mounted his trusty steed, and set off on his noble mission.

Through treacherous forests and across perilous rivers, Sir William journeyed, guided by his unwavering determination. Along the way, he encountered many challenges and faced dangerous creatures, but his courage never wavered.

Finally, after days of travel, Sir William reached the village that ha